In [60]:
import openai
import datasets
from transformers import AutoTokenizer, DataCollatorForSeq2Seq,\
    AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, HfArgumentParser
# from helpers import prepare_dataset_code_summary, compute_rouge_and_bleu
from tqdm import tqdm
import os
import json

In [61]:
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset

In [62]:
from transformers import SummarizationPipeline, pipeline
from torch import Tensor

In [63]:
dataset = datasets.load_dataset('json', data_files="./codesearchnet-corpus/python/final/jsonl/train/python_train_0.jsonl")
dataset = dataset['train']

Found cached dataset json (/Users/ryanchhong/.cache/huggingface/datasets/json/default-ead8d37a6184d20b/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
100%|██████████| 1/1 [00:00<00:00, 907.47it/s]


In [64]:
from transformers import SummarizationPipeline
from torch import Tensor


class CustomSummaryGenerationPipeline(SummarizationPipeline):

    def preprocess(self, inputs):
        max_seq_length = self.tokenizer.model_max_length
        
        examples_with_prompts = list(map(lambda x: "Summarize Python: " + x, inputs))
        # print(examples_with_prompts)
        tokenized_examples = self.tokenizer(
            examples_with_prompts,
            truncation=True,
            max_length=max_seq_length,
            padding='max_length',
            return_tensors = 'pt'
        )
        return tokenized_examples

    def _forward(self, model_inputs):
        outputs = self.model.generate(**model_inputs)
        return {"encoded_pred": outputs}

    def postprocess(self, model_outputs):
        # print(model_outputs)
        encoded_predictions = model_outputs["encoded_pred"]
        decoded_predictions = self.tokenizer.batch_decode(encoded_predictions, skip_special_tokens=True)
        # decoded_predictions = [pred.strip() for pred in decoded_predictions]
        return decoded_predictions

In [65]:
my_pipeline = pipeline(
    task = "summarization",
    model = "./single_line_summarization_model/",
    pipeline_class = CustomSummaryGenerationPipeline
)

In [66]:
store = []
for out in tqdm(my_pipeline(KeyDataset(dataset, 'code')[:2])):
    store.append(out)

100%|██████████| 2/2 [00:00<00:00, 76959.71it/s]


In [68]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [77]:
for line_summary in store:
    line_summary = "\n".join(line_summary)
    gpt_response = openai.Completion.create(
        model="text-davinci-003",
        prompt="Summarize in a couple of sentences:\n\n" + line_summary,
        temperature=0.7,
        max_tokens=64,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    print(gpt_response['choices'][0]['text'].strip())

This code snippet uses the urllib.parse.urlparse function to parse a URL into a dictionary of arguments, then uses the query string from that dictionary to parse into a dictionary of arguments, and finally converts the resulting dictionary into a dictionary of key-value pairs.
This code parses a URL into a dictionary of arguments using the urllib.parse.urlparse library and then parses the URL query string into a dictionary. It also converts a dictionary into a dictionary of key-value pairs.
